<a href="https://colab.research.google.com/github/AI-Lab-2025-2-3rd/ai-project-team-2/blob/main/we_will_rock_you.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import zipfile
from tqdm import tqdm

In [2]:
drive.mount('/content/drive')

zip_path = "/content/drive/Shareddrives/we_will_rock_you/rock_data.zip"
extract_path = "/content/rolling_stones_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting"):
        zip_ref.extract(file, extract_path)

TRAIN_DIR = "/content/rolling_stones_data/open/train"
TEST_DIR = "/content/rolling_stones_data/open/test"

img_size = 224
batch_size = 32
val_split = 0.2

Mounted at /content/drive


Extracting: 100%|██████████| 475038/475038 [04:18<00:00, 1835.71it/s]


In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=val_split
)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = train_gen.num_classes
class_indices = train_gen.class_indices
print("Classes:", class_indices)

Found 304019 images belonging to 7 classes.
Found 76001 images belonging to 7 classes.
Classes: {'Andesite': 0, 'Basalt': 1, 'Etc': 2, 'Gneiss': 3, 'Granite': 4, 'Mud_Sandstone': 5, 'Weathered_Rock': 6}


In [4]:
class_counts = {cls: len(os.listdir(os.path.join(TRAIN_DIR, cls)))
                for cls in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, cls))}

classes = list(class_counts.keys())
counts = list(class_counts.values())
class_labels = np.arange(len(classes))

weights = compute_class_weight(
    class_weight='balanced',
    classes=class_labels,
    y=np.concatenate([np.full(count, i) for i, count in enumerate(counts)])
)
class_weight = {i: w for i, w in enumerate(weights)}
print("class_weight:", class_weight)

class_weight: {0: np.float64(2.024937390099643), 1: np.float64(1.2394085071131782), 2: np.float64(0.5842317986781682), 3: np.float64(1.4605873558226325), 4: np.float64(0.6067999533746681), 5: np.float64(0.7344829319015536), 6: np.float64(3.4068761486395625)}


In [5]:
classes = list(class_counts.keys())
counts = list(class_counts.values())
class_labels = np.arange(len(classes))

weights = compute_class_weight(
    class_weight='balanced',
    classes=class_labels,
    y=np.concatenate([np.full(count, i) for i, count in enumerate(counts)])
)

class_weight = {i: w for i, w in enumerate(weights)}
print("클래스 인덱스:", train_gen.class_indices)
print("class_weight:", class_weight)

클래스 인덱스: {'Andesite': 0, 'Basalt': 1, 'Etc': 2, 'Gneiss': 3, 'Granite': 4, 'Mud_Sandstone': 5, 'Weathered_Rock': 6}
class_weight: {0: np.float64(2.024937390099643), 1: np.float64(1.2394085071131782), 2: np.float64(0.5842317986781682), 3: np.float64(1.4605873558226325), 4: np.float64(0.6067999533746681), 5: np.float64(0.7344829319015536), 6: np.float64(3.4068761486395625)}


In [6]:
class TrainingMonitor(callbacks.Callback):
    def __init__(self, val_data, val_steps):
        super().__init__()
        self.val_data = val_data
        self.val_steps = val_steps

    def on_epoch_end(self, epoch, logs=None):
        val_preds = []
        val_labels = []
        for _ in range(self.val_steps):
            x_batch, y_batch = next(self.val_data)
            preds = self.model.predict(x_batch, verbose=0)
            val_preds.append(np.argmax(preds, axis=1))
            val_labels.append(np.argmax(y_batch, axis=1))
        y_true = np.concatenate(val_labels)
        y_pred = np.concatenate(val_preds)

        cm = confusion_matrix(y_true, y_pred)
        FP = cm.sum(axis=0) - np.diag(cm)
        TN = cm.sum() - (cm.sum(axis=1) + cm.sum(axis=0) - np.diag(cm))
        fpr_per_class = FP / (FP + TN + 1e-7)
        avg_fpr = np.mean(fpr_per_class)

        print(f"Epoch {epoch+1} - val_accuracy: {logs.get('val_accuracy'):.4f}, avg_val_FPR: {avg_fpr:.4f}")


In [7]:
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(img_size, img_size, 3)
)
base_model.trainable = False

inputs = layers.Input(shape=(img_size, img_size, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
initial_epochs = 5
fine_tune_epochs = 5
total_epochs = initial_epochs + fine_tune_epochs

# 20%만 학습
subset_ratio = 0.2
steps_per_epoch = max(1, int(len(train_gen) * subset_ratio))
validation_steps = max(1, int(len(val_gen) * subset_ratio))

monitor_callback = TrainingMonitor(val_gen, validation_steps)

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_initial = model.fit(
    train_gen,
    epochs=initial_epochs,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    class_weight=class_weight,
    callbacks=[monitor_callback]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step - accuracy: 0.5295 - loss: 1.4721Epoch 1 - val_accuracy: 0.4938, avg_val_FPR: 0.0921
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 1289s 663ms/step - accuracy: 0.5295 - loss: 1.4720 - val_accuracy: 0.4938 - val_loss: 1.3928
Epoch 2/5
1477/1900 ━━━━━━━━━━━━━━━━━━━━ 3:00 427ms/step - accuracy: 0.6314 - loss: 1.1828

In [ ]:
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.8)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_gen,
    epochs=total_epochs,
    initial_epoch=history_initial.epoch[-1] + 1,
    validation_data=val_gen,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    class_weight=class_weight,
    callbacks=[monitor_callback]
)